# **Scraping Arxiv Data**

Importing Libraries

In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import concurrent.futures
import re
import pandas as pd
import time

Initializing Threads

In [ ]:
MAX_LINK_THREADS = 30
MAX_PAGE_THREADS = 1000

Creating links from arxiv.org for search page in groups of 1000s (In this case Maths/Year 2021 till September)

In [1]:
master = []
show = 1000
for skip in range(0,31001,1000):
  master.append('https://arxiv.org/list?archive=math&year=21&month=all&skip='+str(skip)+'&show='+str(show))

In [ ]:
abs_links = []

Funcion for scarping paper id for each paper published for a subject

In [2]:
def scrape_links(master_urls):
  source_code=requests.get(master_urls)
  plain_text=source_code.text
  soup=BeautifulSoup(plain_text,'html.parser')
  for a in soup.find_all('a', href=True):
    if 'abs' in a['href']:
      abs_links.append(a['href'])

Using concurrent.futures to create muliple threads for faster scraping

In [ ]:
def download_links(master):
    threads = min(MAX_LINK_THREADS, len(master))
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(scrape_links, master)

Runing the aboove functions

In [ ]:
download_links(master)

In [ ]:
abs_links = set(abs_links)

In [ ]:
all_links = []

Creating links for each paper Ids

In [ ]:
for l in abs_links:
  link = 'https://arxiv.org'+str(l)
  all_links.append(link)

In [ ]:
len(all_links)

0

Scraing each each paper link for the details of a paper

In [ ]:
pages_data = []

def scrape_page_data(page_url):
  source_code=requests.get(page_url)
  plain_text=source_code.text
  soup=BeautifulSoup(plain_text,'html.parser')

  title=soup.find('h1', class_='title')
  unwanted = title.find('span')
  unwanted.extract()

  authors=soup.find('div', class_='authors')
  unwanted = authors.find('span')
  unwanted.extract()

  abstract=soup.find('blockquote', class_='abstract')
  unwanted = abstract.find('span')
  unwanted.extract()

  comments=soup.find('td', class_='comments')
  subject=soup.find('td', class_='subjects')
 
  topic_page_url = page_url
  try:
    topic_title = title.text.strip()
  except:
    topic_title = ''
  try:
    topic_authors = authors.text.strip()
  except:
    topic_authors = ''
  try:
    topic_abstract = abstract.text.strip()
  except:
    topic_abstract = ''
  try:
    topic_comments = comments.text.strip()
  except:
    topic_comments = ''
  try:
    topic_subject = subject.text.strip()
  except:
    topic_subject = ''
  try:
    topic_date = soup.find('strong', text='[v1]').next_sibling.strip()
  except:
    topic_date = ''
  pages_data.append([topic_title,topic_authors,topic_abstract,topic_comments,topic_subject,topic_date,topic_page_url])

Thread management for scaring individual pages

In [ ]:
def page_data_threads(master):
    threads = min(MAX_PAGE_THREADS, len(master))
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(scrape_page_data, master)

In [ ]:
all_links = list(set(all_links))

In [ ]:
len(all_links)

33168

Runing the above functions

In [ ]:
page_data_threads(all_links)

Creating a dataframe

In [ ]:
df = pd.DataFrame(pages_data, columns=['title', 'authors', 'abstract','comments','subject','date','page_url'])

In [ ]:
#df.to_csv('arxiv_math_2021_2109.csv')

Sample Final Data

In [ ]:
df.head()

,title,authors,abstract,comments,subject,date,page_url
0,A new stochastic order based on discrete Lapla...,"Fatemeh Gharari, Masoud Ganji",This paper aims to study a new stochastic orde...,,Statistics Theory (math.ST),"Thu, 8 Apr 2021 12:09:31 UTC (15 KB)",https://arxiv.org/abs/2104.03716
1,Learning Partial Differential Equations in Rep...,George Stepaniants,We propose a new data-driven approach for lear...,"55 pages, 5 figures",Statistics Theory (math.ST); Numerical Analysi...,"Thu, 26 Aug 2021 05:22:24 UTC (4,561 KB)",https://arxiv.org/abs/2108.11580
2,Rigidity theorem of the Bergman kernel by anal...,John Treuer,"In [7], Dong and I proved that the domains $D ...",,Complex Variables (math.CV),"Tue, 5 Jan 2021 06:00:32 UTC (7 KB)",https://arxiv.org/abs/2101.01358
3,Optimization on manifolds: A symplectic approach,"Guilherme França, Alessandro Barp, Mark Girola...",There has been great interest in using tools f...,,Statistical Mechanics (cond-mat.stat-mech); Op...,"Fri, 23 Jul 2021 13:43:34 UTC (959 KB)",https://arxiv.org/abs/2107.11231
4,Options for a finite group model of quantum me...,Robert A. Wilson,There are four finite groups that could plausi...,v2: added a section showing how to obtain a mo...,Group Theory (math.GR); High Energy Physics - ...,"Tue, 20 Apr 2021 09:51:02 UTC (17 KB)",https://arxiv.org/abs/2104.10165
